<a href="https://colab.research.google.com/github/JuanZapa7a/semiotics/blob/main/QAT_MNIST_NOISY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Installation of Larq and necessary dependencies

In [ ]:
!pip -q install tensorflow==2.10.0
!pip -q install larq==0.13.1

import tensorflow as tf
import larq as lq